### 1.Use the Notebook to build the code to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
!conda install bs4 --yes
from bs4 import BeautifulSoup
from urllib.request import urlopen
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
print('import done')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         167 KB
    bs4-4.9.0                  |                0           9 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    soupsieve-2.0              |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.9.0-py36_0
  bs4                pkgs/main/noarch::bs4-4.9.0-0
  soupsieve          pkg

Obtain table data from Wikipedia page using the BeautifulSoup package.

In [2]:
import urllib.request
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as f:
    toronto_N = BeautifulSoup(f)

    
match = toronto_N.find('table', class_='wikitable sortable')

import pandas as pd
import numpy as np
rows = match.find_all('tr')


dt_torontotext=[]

for r in range(len(rows)):
    dt_toronto = rows[r].find_all('td')
    temp=[]
    for val in dt_toronto:
        temp.append(val.text[:-1])
    dt_torontotext.append(temp)    


 Abstract three columns from postal codes table: PostalCode, Borough, and Neighborhood

In [3]:
col =  match.find_all('th')
column = [value.text[:-1] for value in col]
print(column) 


['Postal code', 'Borough', 'Neighborhood']


Transform the data into a pandas dataframe.

In [4]:
df = pd.DataFrame(dt_torontotext, columns = column)
df = df.drop(0)
df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Process the cells that have an assigned borough. Drop cells with a borough that is Not assigned.
Use the .shape method to print the number of rows of your dataframe.

In [5]:
df_new = df.loc[df['Borough']!='Not assigned']
df_n = df_new.reset_index(drop=True)
print(df_n.head())
print(df_n.shape)

  Postal code           Borough                                  Neighborhood
0         M3A        North York                                     Parkwoods
1         M4A        North York                              Victoria Village
2         M5A  Downtown Toronto                    Regent Park / Harbourfront
3         M6A        North York             Lawrence Manor / Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government
(103, 3)


Check the existance of Not assigned data in neighborhood column. 

In [6]:

for i in df['Neighborhood']:
    if i=='Not assigned':
        print(i)   
print('Neighborhood all assigned')     
 

Neighborhood all assigned


### 2. Get the latitude and the longitude coordinates of each neighborhood.

Find geographical coordinates of each postal code from http://cocl.us/Geospatial_data

In [7]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
 
df.rename(columns={'Postal Code':'Postal code'}, inplace=True)
print(df.head())

  Postal code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [8]:
pd_cor = pd.merge(left=df_n, right=df, how='left', left_on='Postal code', right_on='Postal code')
print(pd_cor)
print(pd_cor.shape)


    Postal code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                           Regent Park / Harbourfront  43.654260 -79.360636  
3                    Lawrence Manor / Lawrence Heights  43.718518 -79.464763  
4         Queen's Park / Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98

In [14]:
### 3.Explore the Neiborhoods in Downtown Toronto

You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. to generate maps to visualize your neighborhoods and how they cluster together.

In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium


Solving environment: done

# All requested packages already installed.



 Select boroughs that contain the word Toronto

In [15]:
df_cor= pd_cor[pd_cor['Borough'].str.contains('Toronto')]
df_cor.reset_index(drop=True)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259


In [14]:
neigh = np.array(df_cor['Neighborhood'])
neighb=[]
for i in neigh:
    neighb2 = i.split('/')
    for j in neighb2:
        neighb.append(j)
     
column_names = ['Neighborhood','Postal','Borough','Latitude','Longitude'] 

neighborhoods_t = pd.DataFrame(columns=column_names)


for data in range(len(neighb)):
    nei_bor = df_cor[df_cor['Neighborhood'].str.contains(neighb[data])]
    for i in range(len(nei_bor)):
        row = nei_bor.iloc[i]
        neighborhoods_t = neighborhoods_t.append(
            {'Borough':row['Borough'], 
             'Postal':row['Postal code'],
            'Neighborhood':neighb[data], 
            'Latitude':row['Latitude'], 
            'Longitude':row['Longitude']}, 
            ignore_index=True)

print(neighborhoods_t)

                             Neighborhood Postal           Borough   Latitude  \
0                            Regent Park     M5A  Downtown Toronto  43.654260   
1                            Harbourfront    M5A  Downtown Toronto  43.654260   
2                            Harbourfront    M5V  Downtown Toronto  43.628947   
3                           Queen's Park     M7A  Downtown Toronto  43.662301   
4           Ontario Provincial Government    M7A  Downtown Toronto  43.662301   
..                                    ...    ...               ...        ...   
75                            Cabbagetown    M4X  Downtown Toronto  43.667967   
76                  First Canadian Place     M5X  Downtown Toronto  43.648429   
77                       Underground city    M5X  Downtown Toronto  43.648429   
78                   Church and Wellesley    M4Y  Downtown Toronto  43.665860   
79  Business reply mail Processing CentrE    M7Y      East Toronto  43.662744   

    Longitude  
0  -79.3606

In [16]:
CLIENT_ID = 'FU3FNGEIK2IW5VS01MZC1ZVR2W20LSCEZOUCIDBB2WOU051W' # your Foursquare ID
CLIENT_SECRET = '3DCGCPEJ4YIEDGTY5QMLI3MPS3CLWLA2SOZRIOU2OUGOPCMS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
#neighborhhod_latitude = neighborhoods_t.loc[0,'Latitude']
#neighborhhod_longitude = neighborhoods_t.loc[0,'Longitude']
#neighborhood_name = neighborhoods_t.loc[0,'Neighborhood']


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500  # Define radius
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
    #CLIENT_SECRET, 
    #neighborhhod_latitude, 
    #neighborhhod_longitude, 
    #VERSION, 
    #radius, 
    #LIMIT)
#venue_data = requests.get(url).json()

#venue_data 


From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.function that extracts the category of the venue

In [20]:
#def get_category_type(row):
    #try:
        #categories_list = row['categories']
    #except:
        #categories_list = row['venue.categories']
        
    #if len(categories_list) == 0:
        #return None
    #else:
        #return categories_list[0]['name']

#Now we are ready to clean the json and structure it into a *pandas* dataframe.
#venues = venue_data['response']['groups'][0]['items']
    
#nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
#filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
#nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#print(nearby_venues.head())
#nearby_venues.shape

                     name           categories        lat        lng
0        Roselle Desserts               Bakery  43.653447 -79.362017
1           Tandem Coffee          Coffee Shop  43.653559 -79.361809
2      Morning Glory Cafe       Breakfast Spot  43.653947 -79.361149
3  Cooper Koo Family YMCA  Distribution Center  43.653249 -79.358008
4     Body Blitz Spa East                  Spa  43.654735 -79.359874


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


(48, 4)

In [18]:
# Let's create a function to repeat the same process to all the neighborhoods in Toronto.
def getNearbyVenues(names,latitudes, longitudes, radius=500):
    
    venues_list=[]
    

    for name, lat, lng in zip(names, latitudes, longitudes):
        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
            
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=df_cor['Neighborhood'],
                                 latitudes=df_cor['Latitude'],
                                 longitudes=df_cor['Longitude'])

print(toronto_venues)

# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()





                               Neighborhood  Neighborhood Latitude  \
0                Regent Park / Harbourfront              43.654260   
1                Regent Park / Harbourfront              43.654260   
2                Regent Park / Harbourfront              43.654260   
3                Regent Park / Harbourfront              43.654260   
4                Regent Park / Harbourfront              43.654260   
...                                     ...                    ...   
1618  Business reply mail Processing CentrE              43.662744   
1619  Business reply mail Processing CentrE              43.662744   
1620  Business reply mail Processing CentrE              43.662744   
1621  Business reply mail Processing CentrE              43.662744   
1622  Business reply mail Processing CentrE              43.662744   

      Neighborhood Longitude                      Venue  Venue Latitude  \
0                 -79.360636           Roselle Desserts       43.653447   
1        

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,18,18,18,18,18,18
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [20]:
# Analyze Each Neighborhood
#one hot encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Add neighborhood column back to datafrome
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.head())
#Group rows by neighborhood and by taking the mea of the frequency of occurrence of each category.
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.head())


   Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  ...  Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0       

In [21]:

#Sort the venues in decending order and create new dataframe and display the toop 10 venues for each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10
indicators = ['st','nd','rd']
#create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
#create a create dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']      

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)

print(neighborhoods_venues_sorted.head())

                                        Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1     Brockton / Parkdale Village / Exhibition Place                  Café   
2              Business reply mail Processing CentrE           Yoga Studio   
3  CN Tower / King and Spadina / Railway Lands / ...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar              Beer Bar           Cheese Shop   
1           Coffee Shop             Nightclub        Breakfast Spot   
2         Auto Workshop                  Park            Comic Shop   
3        Airport Lounge      Airport Terminal      Sculpture Garden   
4                  Café    Italian Restaurant        Sandwich Place   

  5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
0            Restaurant     

Cluster Neighborhood

In [22]:
#set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#run k-means clustering
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
#check cluster labels generated for each row n the dataframe
kmeans.labels_[0:10]
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
toronto_merged = df_cor
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
print(toronto_merged.head())

   Postal code           Borough  \
2          M5A  Downtown Toronto   
4          M7A  Downtown Toronto   
9          M5B  Downtown Toronto   
15         M5C  Downtown Toronto   
19         M4E      East Toronto   

                                    Neighborhood   Latitude  Longitude  \
2                     Regent Park / Harbourfront  43.654260 -79.360636   
4   Queen's Park / Ontario Provincial Government  43.662301 -79.389494   
9                       Garden District, Ryerson  43.657162 -79.378937   
15                                St. James Town  43.651494 -79.375418   
19                                   The Beaches  43.676357 -79.293031   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
2                0           Coffee Shop                  Park   
4                0           Coffee Shop      Sushi Restaurant   
9                0        Clothing Store           Coffee Shop   
15               0                  Café           Coffee Shop   
19      

In [23]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
The

In [24]:
#Visualize the resulting clusters
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=11)
#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories,

In [25]:

cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Regent Park / Harbourfront,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Restaurant,Chocolate Shop
4,Queen's Park / Ontario Provincial Government,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Burrito Place,Beer Bar,Italian Restaurant,Juice Bar,Sandwich Place,Burger Joint
9,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant
15,St. James Town,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Hotel,Restaurant,Gym,Italian Restaurant,Department Store
19,The Beaches,0,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
20,Berczy Park,0,Coffee Shop,Cocktail Bar,Beer Bar,Cheese Shop,Restaurant,Café,Bakery,Seafood Restaurant,Bistro,Jazz Club
24,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Middle Eastern Restaurant,Bar,Thai Restaurant,Burger Joint,Fried Chicken Joint
25,Christie,0,Grocery Store,Café,Park,Candy Store,Athletics & Sports,Restaurant,Italian Restaurant,Diner,Nightclub,Coffee Shop
30,Richmond / Adelaide / King,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Gym,Clothing Store,Hotel,Bakery,Steakhouse
31,Dufferin / Dovercourt Village,0,Bakery,Pharmacy,Supermarket,Music Venue,Bank,Brewery,Pool,Recording Studio,Café,Middle Eastern Restaurant
